# HW3

In [81]:
import requests
import pandas as pd

## 1

In [85]:
# Read the destinations from the text file
with open('dests.txt', 'r') as file:
    destinations = file.read().splitlines()

# Initialize empty lists to store the data
distances = []
durations = []
latitudes = []
longitudes = []

# API endpoint URLs
distance_api_url = 'https://maps.googleapis.com/maps/api/distancematrix/json'
geocode_api_url = 'https://maps.googleapis.com/maps/api/geocode/json'
api_key = 'Replace with your own API key'

# API parameters
params = {
    'units': 'metric',
    'origins': 'Tel Aviv',
    'key': api_key 
}

# Iterate over each destination
for destination in destinations:
    # Set the destination parameter
    params['destinations'] = destination
    
    try:
        # Send request to Distance Matrix API
        response = requests.get(distance_api_url, params=params)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                data = response.json()
                # Retrieve the distance and duration values
                distance = data['rows'][0]['elements'][0]['distance']['value']/1000
                duration = data['rows'][0]['elements'][0]['duration']['value']/60
    
                duration_hours = int(duration// 60)  # Calculate the whole number of hours
                duration_minutes = round(duration%60)  # Calculate the remaining minutes

    
                # Append the values to the respective lists
                distances.append(distance)
                durations.append(f"{duration_hours} hours {duration_minutes} minutes")
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    
    # Send request to Geocoding API
    geocode_params = {
        'address': destination,
        'key': api_key  
    }
    try:
        geocode_response = requests.get(geocode_api_url, params=geocode_params)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                geocode_data = geocode_response.json()
    
                # Retrieve the latitude and longitude values
                latitude = geocode_data['results'][0]['geometry']['location']['lat']
                longitude = geocode_data['results'][0]['geometry']['location']['lng']
    
                # Append the values to the respective lists
                latitudes.append(latitude)
                longitudes.append(longitude)
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    

# Create a dataframe with the collected data
data = {
    'Target': destinations,
    'Distance_km': distances,
    'Duration (hour+minutes)': durations,
    'Longitude': longitudes,
    'latitude': latitudes
}
df = pd.DataFrame(data)


In [86]:
df

,Target,Distance_km,Duration (hour+minutes),Longitude,latitude
0,Istanbul,1815.227,21 hours 3 minutes,28.978359,41.008238
1,Amsterdam,4532.541,48 hours 3 minutes,4.904139,52.367573
2,Valletta,3792.882,50 hours 53 minutes,14.514100,35.899237
3,Basel,4118.225,44 hours 10 minutes,7.588576,47.559599
4,Doha,2164.477,22 hours 39 minutes,51.531040,25.285447


## 2

In [87]:
# Sort the dataframe by distance in descending order
sorted_df = df.sort_values(by='Distance_km', ascending=False)

# Get the top 3 cities furthest from Tel Aviv
furthest_cities = sorted_df.head(3)
furthest_cities  = furthest_cities.reset_index(drop=True)
furthest_cities['Target']

0    Amsterdam
1        Basel
2     Valletta
Name: Target, dtype: object